In [6]:
from functions_AIA import *

import os

import matplotlib.pyplot as plt

import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.time import Time
from astropy.visualization import ImageNormalize, SqrtStretch

import sunpy.coordinates  # NOQA
import sunpy.map
from sunpy.net import Fido
from sunpy.net import attrs as a
from functions_AIA import *

import torch

from datetime import timedelta



In [7]:
# Load the csv data containing the events
data = load_data()
data['date'] = pd.to_datetime(data['date'])
data_sorted = data.sort_values(by='date', ascending=True)

In [8]:
data.head()

,#sjh_ID,date,SOL_event,duration,basepoint_X,std_X,basepoint_Y,std_Y,basepoint_X_longitude,basepoint_Y_latitude,max_height,upper_H,lower_H,avg_width,std_width,sigma,flag
0,sjh_2011-01-20T09_1,2011-01-20 09:15:44,SOL2011-01-20T09:00:09L289C165,43.6,-226.577125,7.456468,-956.964375,9.576814,-109.0,-75.0,173.811389,46.785032,-36.862662,52.282512,14.552284,0.417661,1
1,sjh_2011-01-20T23_1,2011-01-20 23:49:20,SOL2011-01-20T22:00:09L353C074,10.8,-143.625000,1.125000,386.404000,2.856000,-8.0,18.0,97.731917,16.548028,-14.151831,83.204500,10.121327,0.337046,10
2,sjh_2011-01-20T23_2,2011-01-20 23:54:32,SOL2011-01-20T22:00:09L353C074,5.6,-142.500000,0.000000,389.260000,0.000000,-8.0,18.0,75.814019,49.693522,-30.017843,40.384010,0.000000,0.635112,110
3,sjh_2011-01-21T04_1,2011-01-21 04:14:32,SOL2011-01-20T22:00:09L353C074,9.2,-100.466000,0.000000,387.028000,0.000000,-6.0,18.0,84.057521,30.622882,-22.445714,79.405020,0.000000,0.462752,10
4,sjh_2011-01-21T06_1,2011-01-21 06:48:56,SOL2011-01-20T22:00:09L353C074,5.6,-11.371400,0.000000,475.010000,0.000000,-0.0,23.0,129.894627,38.593654,-29.753454,73.469787,0.000000,0.405649,110


# Get dataframe of All SOL events and duration

All events in HEK data

In [9]:
timerange = a.Time('2011/01/01 00:00:00', '2017/01/01 00:00:00')
res = Fido.search(timerange, a.hek.CJ) 

Get events analyzed in Zooniverse

In [10]:
Cluster_SOL = data['SOL_event']
Cluster_SOL_unique = Cluster_SOL.unique()
print(Cluster_SOL.shape, Cluster_SOL_unique.shape)

(883,) (229,)


Create dataframe with events in Zooniverse

In [55]:
SOL_events = pd.DataFrame(columns=['event_ID', 'start_time', 'end_time', 'num_jets'])
SOL_events['event_ID'] = Cluster_SOL_unique
SOL_events.set_index('event_ID', inplace=True)
SOL_events.head()

,start_time,end_time,num_jets
event_ID,,,
SOL2011-01-20T09:00:09L289C165,NaN,NaN,NaN
SOL2011-01-20T22:00:09L353C074,NaN,NaN,NaN
SOL2011-01-24T22:15:09L358C011,NaN,NaN,NaN
SOL2011-01-26T01:52:21L007C038,NaN,NaN,NaN
SOL2011-02-13T04:57:46L131C056,NaN,NaN,NaN


Fill the dataframe

In [59]:
for event in SOL_events.index:
    # Extract data for the current cluster
    event_info = res['hek'][res['hek']["SOL_standard"] == event][0]  # take [0] because if several events they all contain same info
    # Extract the start time
    start_time = event_info['event_starttime'].value
    # Extract the end time
    end_time = event_info['event_endtime'].value
    # Store the start and end time in the dataframe
    SOL_events.loc[event, 'start_time'] = start_time
    SOL_events.loc[event, 'end_time'] = end_time

    # Number of jets
    num_jets = len(data[data['SOL_event'] == event])
    SOL_events.loc[event, 'num_jets'] = num_jets



# Convert the start and end time to datetime objects
SOL_events['start_time'] = pd.to_datetime(SOL_events['start_time'])
SOL_events['end_time'] = pd.to_datetime(SOL_events['end_time'])

# Calculate duration of each event
SOL_events['duration'] = SOL_events['end_time'] - SOL_events['start_time']

In [60]:
display(SOL_events.head())
print(SOL_events.shape) 

,start_time,end_time,num_jets,duration
event_ID,,,,
SOL2011-01-20T09:00:09L289C165,2011-01-20 09:00:09,2011-01-20 09:59:33,1,0 days 00:59:24
SOL2011-01-20T22:00:09L353C074,2011-01-20 22:00:09,2011-01-21 16:00:09,5,0 days 18:00:00
SOL2011-01-24T22:15:09L358C011,2011-01-24 22:15:09,2011-01-25 00:00:09,3,0 days 01:45:00
SOL2011-01-26T01:52:21L007C038,2011-01-26 01:52:21,2011-01-26 03:54:45,1,0 days 02:02:24
SOL2011-02-13T04:57:46L131C056,2011-02-13 04:57:46,2011-02-13 05:56:34,2,0 days 00:58:48


(229, 4)


# Simplify dataframe with Jet Events
And add end time for each of them

In [ ]:
jet_events = pd.DataFrame(columns=['event_ID', 'start_time', 'end_time', 'duration'])
jet_events['event_ID'] = data['SOL_event']
jet_events['start_time'] = data['date']
jet_events['duration'] = data['duration']

for i in range(len(jet_events)):
    jet_events.loc[i, 'end_time'] = jet_events.loc[i, 'start_time'] + pd.Timedelta(minutes=jet_events.loc[i, 'duration'])

In [177]:
jet_events.head()

,event_ID,start_time,end_time,duration
0,SOL2011-01-20T09:00:09L289C165,2011-01-20 09:15:44,2011-01-20 09:59:20,43.6
1,SOL2011-01-20T22:00:09L353C074,2011-01-20 23:49:20,2011-01-21 00:00:08,10.8
2,SOL2011-01-20T22:00:09L353C074,2011-01-20 23:54:32,2011-01-21 00:00:08,5.6
3,SOL2011-01-20T22:00:09L353C074,2011-01-21 04:14:32,2011-01-21 04:23:44,9.2
4,SOL2011-01-20T22:00:09L353C074,2011-01-21 06:48:56,2011-01-21 06:54:32,5.6


# Now we get to the weird part, getting ranges without jets

Basically, I create a boolean array where each element is a second in the range of dates

- By default, all elements are False
- I set to True all elements where there is a SOL_events event
- I now set to false all elements where there is a jet jet_events

Now, we are left with an array that is True only for values that are in registered HEK events but not in registered jets

We now transform it back to ranges of dates.

In [151]:
start_time_total = SOL_events.start_time.min()
end_time_total = SOL_events.end_time.max()
total_seconds = int((end_time_total - start_time_total).total_seconds())
time_array = np.zeros(total_seconds, dtype=bool)
total_seconds

187076405

In [156]:
for event in SOL_events.index: 
    start_time = SOL_events.loc[event, 'start_time']
    end_time = SOL_events.loc[event, 'end_time']
    start_index = int((start_time - start_time_total).total_seconds())
    end_index = int((end_time - start_time_total).total_seconds())
    time_array[start_index:end_index] = True

for event in jet_events.index: 
    start_time = jet_events.loc[event, 'start_time']
    end_time = jet_events.loc[event, 'end_time']
    start_index = int((start_time - start_time_total).total_seconds())
    end_index = int((end_time - start_time_total).total_seconds())
    time_array[start_index:end_index] = False

In [166]:
display(time_array)
print(time_array.sum() / len(time_array))

array([ True,  True,  True, ...,  True,  True,  True])

0.01023402710780122


In [168]:
def find_true_ranges(array):
    ranges = []
    start = None

    for i, value in enumerate(array):
        if value and start is None:
            start = i  # Start of a new true range
        elif not value and start is not None:
            ranges.append((start, i - 1))
            start = None

    # Handle case where array ends with True
    if start is not None:
        ranges.append((start, len(array) - 1))

    return ranges


ranges_dates = find_true_ranges(time_array)

In [189]:
ranges_dates_df = pd.DataFrame(ranges_dates, columns=['start_time', 'end_time'])

In [190]:
ranges_dates_df.head()

,start_time,end_time
0,0,934
1,3551,3563
2,46800,53350
3,53999,69262
4,69815,78526


Just to be safe, let's add one minute of margin

In [193]:
ranges_dates_df['start_time'] = ranges_dates_df['start_time'] + 60
ranges_dates_df['end_time'] = ranges_dates_df['end_time'] - 60

# Remove rows where the start time is after the end time
ranges_dates_df = ranges_dates_df[ranges_dates_df['start_time'] < ranges_dates_df['end_time']]

In [194]:
dates_no_jets = pd.DataFrame(columns=['start_time', 'end_time', 'duration'])
dates_no_jets['start_time'] = ranges_dates_df['start_time'].apply(lambda x: start_time_total + pd.Timedelta(seconds=x))
dates_no_jets['end_time'] = ranges_dates_df['end_time'].apply(lambda x: start_time_total + pd.Timedelta(seconds=x))
dates_no_jets['duration'] = dates_no_jets['end_time'] - dates_no_jets['start_time']

In [195]:
dates_no_jets.head()

,start_time,end_time,duration
0,2011-01-20 09:01:09,2011-01-20 09:14:43,0 days 00:13:34
2,2011-01-20 22:01:09,2011-01-20 23:48:19,0 days 01:47:10
3,2011-01-21 00:01:08,2011-01-21 04:13:31,0 days 04:12:23
4,2011-01-21 04:24:44,2011-01-21 06:47:55,0 days 02:23:11
5,2011-01-21 06:55:32,2011-01-21 11:49:31,0 days 04:53:59


# Keep only ranges with a specific duration
Longer than the required 30 images, 24 seconds between images

In [197]:
num_image = 30
seconds_images = (num_image-1) * 24

dates_no_jets_long = dates_no_jets[dates_no_jets['duration'] >= pd.Timedelta(seconds=seconds_images)]

In [200]:
display(dates_no_jets_long.head())
print(dates_no_jets_long.shape)

,start_time,end_time,duration
0,2011-01-20 09:01:09,2011-01-20 09:14:43,0 days 00:13:34
2,2011-01-20 22:01:09,2011-01-20 23:48:19,0 days 01:47:10
3,2011-01-21 00:01:08,2011-01-21 04:13:31,0 days 04:12:23
4,2011-01-21 04:24:44,2011-01-21 06:47:55,0 days 02:23:11
5,2011-01-21 06:55:32,2011-01-21 11:49:31,0 days 04:53:59


(451, 3)


In [201]:
display(dates_no_jets.head())
print(dates_no_jets.shape)

,start_time,end_time,duration
0,2011-01-20 09:01:09,2011-01-20 09:14:43,0 days 00:13:34
2,2011-01-20 22:01:09,2011-01-20 23:48:19,0 days 01:47:10
3,2011-01-21 00:01:08,2011-01-21 04:13:31,0 days 04:12:23
4,2011-01-21 04:24:44,2011-01-21 06:47:55,0 days 02:23:11
5,2011-01-21 06:55:32,2011-01-21 11:49:31,0 days 04:53:59


(679, 3)
